### Scenario 2: A cross-functional team with one data scientist working on an ML model
MLflow setup:

 - tracking server: yes, local server
 - backend store: sqlite database
 - artifacts store: local filesystem


The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

```mlflow server --backend-store-uri sqlite:///mlflow.db```

In [1]:

import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:

mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1765287936222, experiment_id='2', last_update_time=1765287936222, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1765120390682, experiment_id='1', last_update_time=1765120390682, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1765120032098, experiment_id='0', last_update_time=1765120032098, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]

In [4]:

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn
import pandas as pd

# Set the experiment name
mlflow.set_experiment("my-experiment-2")

with mlflow.start_run():

    # Load data
    data = load_iris(as_frame=True)  # returns a pandas DataFrame and a Series
    X = data.data                    # DataFrame with proper column names
    y = data.target                  # Series with labels

    # Log model parameters
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    # Train the model
    lr = LogisticRegression(**params).fit(X, y)

    # Predict and log accuracy
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    # Log the model with input example (to generate signature)
    mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="models",
        input_example=X.iloc[[0]]  # Helps auto-infer input signature
    )

    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/12/10 15:28:26 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-2' does not exist. Creating a new experiment.
2025/12/10 15:28:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
d:\MLOPS\mlops-zoomcamp-course\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.wa

default artifacts URI: 'mlflow-artifacts:/3/13989e4aab27479ca65adeb2340c7ee5/artifacts'
🏃 View run respected-flea-87 at: http://127.0.0.1:5000/#/experiments/3/runs/13989e4aab27479ca65adeb2340c7ee5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


### Interacting with the model registry

In [5]:

from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [6]:

client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1765149383257, deployment_job_id='', deployment_job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', description='', last_updated_timestamp=1765150652713, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1765150652713, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1765150652713, metrics=None, model_id=None, name='xgbooxtmodel', params=None, run_id='3f17e2d45c1a4fdf8db9aca8b993a0e6', run_link='', source='models:/m-186a8c7212ac4458a6885488ce71b059', status='READY', status_message=None, tags={}, user_id='', version='2'>], name='xgbooxtmodel', tags={}>]

In [7]:

run_id = client.search_runs(experiment_ids='1')[0].info.run_id

mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.


MlflowException: Unable to find a logged_model with artifact_path models under run 3f17e2d45c1a4fdf8db9aca8b993a0e6